In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
size(d) # Should be 504x8

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(504, 8)

pulled_left, condition, prosoc_left

In [2]:
@model m10_3(y, x₁, x₂) = begin
    α ~ Normal(0, 10)
    βp ~ Normal(0, 10)
    βpC ~ Normal(0, 10)

    for i ∈ 1:length(y)
        p = logistic(α + (βp + βpC * x₁[i]) * x₂[i])
        y[i] ~ Binomial(1, p)
    end
end;

posterior = sample(m10_3(d[:,:pulled_left], d[:,:condition], d[:,:prosoc_left]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.2
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.059976435076812704, std = [1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 350.02213274299993;
  #lf / sample        = 0.0;
  #evals / sample     = 31.278;
  pre-cond. metric    = [1.0, 1.0, 1.0].


Fix the inclusion of adaptation samples

In [3]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

Rethinking result

In [4]:
m_10_03t_result = "
      Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a    0.05   0.13      -0.15       0.25  3284    1
 bp   0.62   0.22       0.28       0.98  3032    1
 bpC -0.11   0.26      -0.53       0.29  3184    1
";

Describe the draws

In [5]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                  SD                       Naive SE                      MCSE               ESS   
       α    0.049975465  0.12762142607358978318999 0.0040357438463134330183801 0.0046943050014495038904649 739.10234
     βpC   -0.088285519  0.26110545317781258534140 0.0082568794153233746047071 0.0160372904152901452068392 265.07569
  lf_num    0.000000000  0.00000000000000000000000 0.0000000000000000000000000 0.0000000000000000000000000       NaN
 elapsed    0.175691735  0.08643314501364857127541 0.0027332560357475484645595 0.0042715156684473998965901 409.44599
 epsilon    0.059976435  0.00000000000000006942366 0.0000000000000000021953689 0.0000000000000000023129646 900.90090
      βp    0.603323867  0.22304769749361746944416 0.0070533875093606113529376 0.0132345314887681118204421 284.03924
      lp -349.295271663  1.20982004255240727097487 0.038257868933874360

End of m_10_03t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*